<a href="https://colab.research.google.com/github/MattJCR/CV2-detectar-caras-sonriendo-en-videos/blob/master/CV2_detectar_caras_sonriendo_en_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install --upgrade pytube 
# Importamos las bibliotecas necesarias
import cv2
from PIL import Image
import numpy as np
from google.colab.patches import cv2_imshow
from pytube import YouTube
from IPython.display import Video
from base64 import b64encode
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
# Inicio de la medición de tiempo
start_time = time.time()

In [12]:
# URL del video
video_url = "https://www.youtube.com/watch?v=RENDRigVHWU"
# download the video using pytube
yt = YouTube(video_url)
# Obtener la duración del video en segundos
yt_duration = yt.length
video = yt.streams.get_highest_resolution()

# Descarga el video a un archivo en el directorio actual
url = video.download()
print('Ruta del video:',url)

# Abrimos el video con cv2
video = cv2.VideoCapture(url)

# Abrir el video con una tasa de fps de 24 (Reduce el tiempo de tratamiento al tener menos FPS)
FPS = 24
video.set(cv2.CAP_PROP_FPS, FPS)

# Comprueba si se ha podido abrir el video
if not video.isOpened():
    print("Error al abrir el video")
    exit()

# Creamos una instancia del detector de caras de cv2
#face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml") 
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt.xml") # Da buenos resultados
#face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml")
#face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt_tree.xml")
#face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_upperbody.xml") # No da muy buenos resultados

Ruta del video: /content/Anuncio McDonald´s - Estoy probando cosas nuevas - Publicidad Comercial Spot 2017.mp4


In [13]:
# Ancho y alto del video
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Crea un objeto VideoWriter para guardar el video editado
video_writer = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*"MP4V"), FPS, (width, height))

In [14]:
def is_smiling(face_region):
    '''
    DOC: https://dontrepeatyourself.org/post/smile-detection-with-python-opencv-and-haar-cascade/
    '''
    # Cargar el clasificador de sonrisas
    smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

    # Detectar sonrisas en la región de la cara
    #smiles = smile_cascade.detectMultiScale(face_region, 1.3, 8)

    # Aplica el detector de sonrisa a la face_region
    smile_rects, rejectLevels, levelWeights = smile_cascade.detectMultiScale3(face_region, 1.0355, 25, outputRejectLevels=True)
    smiles = False
    # Si no se detectó nada, consideramos esto como una detección de "Not Smiling"
    if len(levelWeights) == 0:
        smiles = False
    else:
        # Si 'levelWeights' es inferior a 2, clasificamos esto como "Not Smiling"
        if max(levelWeights) < 2:
            csmiles = False
        # De lo contrario, hay una sonrisa en la face_region
        else:
            smiles = True
    # Retornar verdadero si se detecta al menos una sonrisa, falso en caso contrario
    return smiles


In [15]:
# Vamos a calcular el procentaje de "smiling" que tenemos en el video
total_video = 0
total_smile = 0

In [16]:
# Iteramos sobre cada frame del video
while True:
    total_video += 1
    # Leemos el frame actual del video
    ret, frame = video.read()

    # Si es el último frame, salimos del bucle
    if not ret:
      print('Video end')
      break

    # Convertimos el frame a una imagen de PIL
    frame_pil = Image.fromarray(frame)
    frame_pil = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_BGR2GRAY)
    # Detectamos las caras en la imagen usando el detector
    faces = face_cascade.detectMultiScale(
          frame_pil,
          scaleFactor = 1.0625,
          minNeighbors = 8
    )
    # print(frame_pil.shape)
    # Dibujamos un rectángulo en cada cara detectada
    for (x, y, w, h) in faces:
        # Determinar si la cara está sonriendo
        smile = is_smiling(frame_pil[y:y+h, x:x+w])
        color = (0,255,0) if smile else (0,0,255)
        if smile: total_smile += 1
        # Dibujar un rectángulo alrededor de la cara y escribir si está sonriendo
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, "Smiling" if smile else "Not smiling", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    # Escribe el frame editado en el archivo de salida
    video_writer.write(frame)

# Cerramos el video original
video.release()
# Cierra el archivo de salida
video_writer.release()

Video end


In [17]:
# Reproducir el video
from IPython.display import Video
video_file = './output.mp4'
Video(video_file, width=640, height=480)
 

In [18]:
# Final de la medición de tiempo
end_time = time.time()

# Cálculo del tiempo de ejecución en minutos
elapsed_time_minutes = (end_time - start_time) / 60

# Mostrar el tiempo de ejecución en minutos
print("Tiempo de ejecución:", elapsed_time_minutes, "minutos")

# Mostrar la duración del video
print("Duración del video:", yt_duration / 60, "minutos")

print('Smiling:',str(total_smile/total_video) + '%')

Tiempo de ejecución: 11.641462389628092 minutos
Duración del video: 0.5166666666666667 minutos
Smiling: 0.14790575916230367%
